In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load("en")
doc = nlp("the project team must demonstrate mockups of UI changes to project stakeholders early in the development process”. Such an example comprises two actors: “project team")
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
#Installing dependencies and upgrading spaCy version 
!pip install -U spacy transformers
!spacy download en_core_web_trf
!spacy download en_core_web_lg
!pip install owlready2

     |████████████████████████████████| 6.0 MB 4.2 MB/s 
     |████████████████████████████████| 3.1 MB 24.6 MB/s 
     |████████████████████████████████| 451 kB 51.4 MB/s 
     |████████████████████████████████| 628 kB 50.0 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 181 kB 56.5 MB/s 
     |████████████████████████████████| 10.1 MB 26.3 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 34.2 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: pyyaml
    Found existing inst

In [ ]:
import spacy 
#necessary files for loading and running relex_model
import sys
sys.path.append('/content/drive/MyDrive/thesis_models/relex/')
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

#loading the NER model
ner_model = spacy.load("/content/drive/MyDrive/thesis_models/rener/model-best")
#loading the relationshiop extraction model
relex_model = spacy.load("drive/MyDrive/RENER/latest_models_v2/relex/model-best")

/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.7/dist-packages/spacy_transformers/pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
#base language model
nlp = spacy.load("en_core_web_lg")

In [ ]:
#Testing ner and relex
from spacy import displacy
doc = ner_model("as an admin i want to change the user password and username from the main page")
#doc = ner_model("as an investor, i want to invest into real state, cryptocurrencies, and old cars")
displacy.render(doc, style = "ent", jupyter=True)
relationships = []
for name, proc in relex_model.pipeline:
  doc = proc(doc)
  for value, rel_dict in doc._.rel.items():
    for e1 in doc.ents:
      for e2 in doc.ents:
        if e1.start == value[0] and e2.start == value[1]:
          new_rel_dict = {"hasAction":rel_dict["Actor"], "hasRelatedConcept":rel_dict["Object"], "hasAttribute":rel_dict["Attribute"]}
          relationships.append((e1.text, e2.text, new_rel_dict))
          print(f" entities: {e1.text, e2.text} ---> predicted relation: {new_rel_dict}")

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


 entities: ('admin', 'change') ---> predicted relation: {'hasAction': 0.9915837, 'hasRelatedConcept': 0.11627535, 'hasAttribute': 0.00022454842}
 entities: ('admin', 'user') ---> predicted relation: {'hasAction': 0.059147686, 'hasRelatedConcept': 7.030888e-06, 'hasAttribute': 0.105927266}
 entities: ('admin', 'password') ---> predicted relation: {'hasAction': 0.0045126774, 'hasRelatedConcept': 0.0005600151, 'hasAttribute': 0.015889741}
 entities: ('admin', 'username') ---> predicted relation: {'hasAction': 0.0052017057, 'hasRelatedConcept': 0.0004532704, 'hasAttribute': 0.010382948}
 entities: ('admin', 'main page') ---> predicted relation: {'hasAction': 0.05296609, 'hasRelatedConcept': 1.4336067e-07, 'hasAttribute': 0.0008534355}
 entities: ('change', 'admin') ---> predicted relation: {'hasAction': 0.9947778, 'hasRelatedConcept': 0.14989226, 'hasAttribute': 0.00021082172}
 entities: ('change', 'user') ---> predicted relation: {'hasAction': 0.0012514348, 'hasRelatedConcept': 0.41990322

In [ ]:
### SQUARE
def preprocess(corpus_path, nlp_model):
  #chunk into "paragraphs" ---> separator: breakline
  with open(corpus_path, 'r') as f:
    corpus_data = f.read()

  corpus_data = corpus_data.split("\n")
  corpus_data = [text for text in corpus_data if text != ""]

  return corpus_data

def get_rel(doc):
  #doc: ner_model(doc)
  relationships = []
  for name, proc in relex_model.pipeline:
    doc = proc(doc)
    for value, rel_dict in doc._.rel.items():
      for e1 in doc.ents:
        for e2 in doc.ents:
          if e1.start == value[0] and e2.start == value[1]:
            relationships.append((e1.text, e2.text, max([(key, rel_dict[key]) for key in rel_dict.keys()], key=lambda x: x[1])))
  return relationships

def preprocess_corpus(corpus_path, ner_model, relex_model, nlp_model):
  corpus_data = preprocess(corpus_path, nlp_model)
  ner_corpus = [doc for doc in ner_model.pipe(corpus_data)]
  nlp_corpus = [doc for doc in nlp_model.pipe(corpus_data, disable=["ner"])]
  relex_corpus = []
  for doc in ner_corpus:
    relex_corpus.append(get_rel(doc))

  return corpus_data, ner_corpus, relex_corpus, nlp_corpus

In [ ]:
def get_candidate_docs(pp_focus, docs, threshold):
  doc_similarity = []
  docs = list(docs)
  for idx, doc in enumerate(docs):
    doc_similarity.append((idx, pp_focus.similarity(doc)))
  #candidate_docs = [docs[doc[0]].text for doc in doc_similarity if doc[1] > threshold]
  candidate_docs = [doc[0] for doc in doc_similarity if doc[1] > threshold]

  return candidate_docs

def extract_answer(question_data, ner_corpus, relex_corpus, nlp_corpus):
  feature_rel_dict = {"FUNCTION":["Actor"],
                      "CONCEPT":["Attribute", "Object"],
                      "ATTRIBUTE":["Attribute", "Object"]}  
  answer_type_dict = {"ATTRIBUTE":["OBJECT", "ACTOR"],
                      "ACTOR":["ACTOR"],
                      "ACTION":["ACTION"],
                      "OBJECT":["OBJECT"]}
  question_text = question_data["text"]
  question_scope = question_data["scope"] #general: whole text; specific: focus based question (semantic similarity)
  question_focus_concept = question_data["focus_concept"]
  answer_type = question_data["focus_feature"]

  #preprocessed_corpus = preprocess(corpus_path, nlp) **corpus_data
   
  answer = []
  if(question_scope == "general"):
    #labeling entities
    #preprocessed_corpus = ner_model.pipe(preprocessed_corpus) ** ner_corpus

    for chunk in ner_corpus:
      for entity in chunk.ents:
        if(entity.label_ in answer_type_dict[answer_type.upper()]):
          answer.append(entity.text)
  elif(question_scope == "specific"):
    candidate_docs = get_candidate_docs(nlp(question_focus_concept), nlp_corpus, 0.4)
    print(f"Candidate docs idx: \n ---> {candidate_docs}")
    relex_pp_docs = [doc for idx, doc in enumerate(relex_corpus) if idx in candidate_docs]
    ner_pp_docs = [doc for idx, doc in enumerate(ner_corpus) if idx in candidate_docs]

    for idx, doc in enumerate(ner_pp_docs):
      relationships = relex_pp_docs[idx]

      #print(relationships)
      for entity in doc.ents:
        if(entity.label_ in answer_type_dict[answer_type.upper()]):
          #Special case for functions --> (Function - Object)
          #print(f"answer_type {answer_type}")
          if(answer_type == "function"):
            for rel in relationships:
              if(rel[0]==question_focus_concept and rel[1]==entity.text):
                if(rel[2][0] in feature_rel_dict[answer_type.upper()] and rel[2][1] > 0.5):
                  #searching for function objects
                  for rel2 in relationships:
                    if(rel2[0] == entity.text and rel2[2][0] == "Object" and rel2[2][1] > 0.5):
                      func_object = rel2[1]
                      answer.append(f"{entity.text}*-*{func_object}*-*")
                      #answer.append((entity.text, func_object))
          elif(answer_type == "attribute"):
            rel_1_score, rel_2_score = 0,0
            for rel in relationships:
              if(rel[2][0] in feature_rel_dict[answer_type.upper()] and rel[2][1]>0.5):
                if(rel[0]==question_focus_concept and rel[1]==entity.text):
                  rel_1_score = rel[2][1]
                #Checking the opposite direction relationship between the same concepts
                #Just return it if is greater  
                elif(rel[0]==entity.text and rel[1]==question_focus_concept):
                  rel_2_score = rel[2][1]
            if(rel_1_score > rel_2_score):
              answer.append(f"{entity.text}")
          else:
            for rel in relationships:
              if(rel[0]==question_focus_concept and rel[1]==entity.text):
                #print(rel[2][0], feature_rel_dict[answer_type.upper()])
                #print(rel[2][1])
                if(rel[2][0] in feature_rel_dict[answer_type.upper()] and rel[2][1] > 0.5):
                  answer.append(entity.text)

  return set(answer)


In [ ]:
import re
def get_question_focus(question):
  import inflect
  inf = inflect.engine()
  get_singular = lambda x: inf.singular_noun(x)
  get_plural = lambda x: inf.plural(x)

  #Pattern
  focus_matches = re.search('^what (are|is) the ([a-zA-Z].*) (of|in)\s?(the)?\s?(\"[-\[\]\*\?!#$%&a-zA-Z0-9].*\")', question)
  focus_feature = focus_matches.group(2)
  focus_concept = focus_matches.group(5)
  focus_concept = focus_concept.replace('"', '')

  feature_singular = get_singular(focus_feature) 
  focus_feature =  focus_feature if feature_singular == False else feature_singular #focus head
  return focus_concept, focus_feature 
  
get_question_focus('what are the questions of the "candidates"?')

In [ ]:
#Classes-attributes(object and data properties)
def get_ontology_structure(ontology):

  #Inherited attributes
  def add_inherited_attributes():
    for ont_class in ontology_classes:
      class_superclasses = ont_class.is_a
      if(class_superclasses[0].name != "Thing"):
        for superclass in class_superclasses:
          ontology_structure[ont_class.name] += ontology_structure[superclass.name]
    return ontology_structure

  #Removing superclasses
  def remove_superclasses():
    for superclass in superclasses:
      del(ontology_structure[superclass.name])
    return ontology_structure
  
  ontology_classes = [ont_class for ont_class in ontology.classes()]
  ontology_properties = [ont_property for ont_property in ontology.properties()]
  ontology_structure = dict((ont_class.name, []) for ont_class in ontology_classes) #class-attributes
  superclasses = [ont_class for ont_class in ontology.classes() if len(list(ont_class.subclasses())) > 0]

  get_property_range_name = lambda x: "none" if type(x) is type else x.name

  for ont_property in ontology_properties:
    ont_property_features = eval(ont_property.comment[0])
    ont_property_features["range"] = get_property_range_name(ont_property.range[0])
    ontology_structure[ont_property.domain[0].name].append((ont_property.name, ont_property_features))

  #Additional steps
  ontology_structure = add_inherited_attributes()
  #ontology_structure = remove_superclasses()
  
  return ontology_structure

In [ ]:
def generate_question(ontology, ner_corpus, relex_corpus, nlp_corpus, answer_dict=None, answer_dictionary=None, individual="discourse", query_class="Discourse"):
  import re
  import inflect
  from termcolor import colored, cprint
  validation_dictionary = {}
  inflect = inflect.engine()

  #concept-->{"c1":{"type":("actor"/"object"), "attribute":[c2, c3, ..., cn]}}
  #function-->{"f1":("c_actor", "action_verb", "c_n"), ...}
  
  if(answer_dict is None):
    answer_dict = {"discourse":{"concept":{},"function":{}}}

  ontology_structure = get_ontology_structure(ontology)
  if(answer_dictionary is None):
    answer_dictionary = dict((key, []) for key in ontology_structure.keys())
  
  beautify_property_str = lambda x: " ".join(re.findall("[A-Z][^A-Z]*", x[3:])).lower()
  beautify_individual_str = lambda x: " ".join(individual.split("_"))

  #First question ---> System 'has-a' relationships (default)
  wh_auxiliaries = {"single":"is", "multiple":"are"}
  ontology_classes = list(ontology_structure.keys())

  for ont_property in ontology_structure[query_class]:
    property_name, property_features = ont_property
    property_name = beautify_property_str(property_name)
    is_property_askable, property_cardinality, property_range = map(lambda x: property_features[x], 
                                                                              property_features.keys())
    property_name = inflect.plural(property_name) if property_cardinality == "multiple" else property_name

    question = "what %s the %s %s \"%s\"? " % (wh_auxiliaries[property_cardinality], 
                                               property_name,
                                               "in" if property_name == "related concepts" else "of the", 
                                               individual)
    #question = input()
    printing_question = question.replace("*-*", " ").strip()

    if(is_property_askable):
      #printing only askable questions
      print("*",colored(printing_question, attrs=['bold', 'dark']), colored("---> askable", "green") if is_property_askable else colored("---> not askable", "red"))
      
      #related_concept case --> *-* func *-* obj
      patt_match = question.find("*-*")
      if(patt_match != -1):
        question_copy = question[patt_match+3:]
        right_end = question_copy.find("*-*")
        focus_concept = question_copy[:right_end].strip()
        focus_feature = "related concept"
        #print(f"{focus_concept} xxx {focus_feature}")
      else:
        focus_concept, focus_feature = get_question_focus(question)
        #print("Focus feature: %s --- Focus concept: %s"%(focus_feature, focus_concept))

      #Temporal ---- Caso relatedConcept de función
      if(focus_feature == "related concept"):
        answer = [focus_concept]
      else:
        question_data = {"text":question,
                         "scope":"general" if focus_concept == "discourse" else "specific",
                         "focus_concept":focus_concept,
                         "focus_feature":focus_feature}
        #print(f"question data --> {question_data}")
        answer = extract_answer(question_data, ner_corpus, relex_corpus, nlp_corpus)
      validation_dictionary[printing_question] = answer 
      #answer = input()
      if(len(answer)>0):
        printing_answer = list(map(lambda x: x.replace("*-*"," ").strip(), answer))
        print(colored("Answer ---> ", "yellow", attrs=["bold"]), printing_answer)
        #Avoiding deep asking for additional roles.
        if(focus_feature != "additional role"):
          for f_individual in answer:
            #S A C A R a función
            #---- building the dict for populating the ontology
            if(property_range == "Actor"):
              if(answer_dict["discourse"]["concept"].get(f_individual) is None):
                answer_dict["discourse"]["concept"][f_individual] = {"type":"actor", "attribute":[]}
            elif(property_range == "Concept"):
              existent_concept = answer_dict["discourse"]["concept"].get(f_individual)
              if(existent_concept is None): #concept-object type, not actor
                answer_dict["discourse"]["concept"][f_individual] = {"type":"object", "attribute":[]}
            elif(property_range == "Function"): 
              function_parts = f_individual.split("*-*")
              f_action_verb = function_parts[0]
              f_related_concept = function_parts[1]
              answer_dict["discourse"]["function"][f"{individual}**{f_action_verb}**{f_related_concept}"] = {"f_related_actor":individual,
                                                                                                             "f_action_verb":f_action_verb,
                                                                                                             "f_related_concept":f_related_concept}
            if(property_name == "attributes"):
              answer_dict["discourse"]["concept"][individual]["attribute"].append(f_individual)
            #----

            if(f_individual not in answer_dictionary[property_range]):
              answer_dictionary[property_range].append(f_individual)
              #-> preventing question repetition 
              #questions based on the property type
              if(property_name in ontology_classes):
                generate_question(ontology, ner_corpus, relex_corpus, nlp_corpus, answer_dict=answer_dict, answer_dictionary=answer_dictionary, individual=f_individual, query_class=property_name)
              #questions based on the range type
              elif(property_range not in ["none", ""]):
                generate_question(ontology, ner_corpus, relex_corpus, nlp_corpus, answer_dict=answer_dict, answer_dictionary=answer_dictionary, individual=f_individual, query_class=property_range)
      else:
        print(colored("Answer ---> ", "yellow", attrs=["bold"]), "None")
  return answer_dictionary, answer_dict, validation_dictionary

In [ ]:
%xmode Verbose
from google.colab import files

def run_quare():
  #Basic corpus composing
  corpus_path = "blit_draft_comprimido.txt"
  corpus_data, ner_corpus, relex_corpus, nlp_corpus = preprocess_corpus(corpus_path, ner_model, relex_model, nlp)

  ontology_path = "drive/MyDrive/RENER/answer_extraction/ontre.owl"
  ontology = get_ontology("file://"+ontology_path).load()
  quare = generate_question(ontology, ner_corpus, relex_corpus, nlp_corpus)
  ontre_data = []
  #ontre_data = quare[1] #answer_dictionary 2.0 (built and organized from answers)
  #print(ontre_data)
  #populate_ontology(ontology, ontre_data, ontology_path)
  #files.download(ontology_path)
  return ontology, ontre_data, ontology_path
ontology, ontre_data, ontology_path = run_quare()

In [ ]:

from owlready2 import *
def get_corpuses(corpus_path):
  corpus_data, ner_corpus, relex_corpus, nlp_corpus = preprocess_corpus(corpus_path, ner_model, relex_model, nlp)
  return corpus_data, ner_corpus, relex_corpus, nlp_corpus

%time corpus_data, ner_corpus, relex_corpus, nlp_corpus = get_corpuses("cask_dataset.txt")

In [ ]:
def get_candidate_docs(pp_focus, docs, threshold):
  doc_similarity = []
  docs = list(docs)
  for idx, doc in enumerate(docs):
    doc_similarity.append((idx, pp_focus.similarity(doc)))
  #candidate_docs = [docs[doc[0]].text for doc in doc_similarity if doc[1] > threshold]
  candidate_docs = [doc[0] for doc in doc_similarity if doc[1] > threshold]

  return candidate_docs

def extract_answer(question_data, ner_corpus, relex_corpus, nlp_corpus):
  feature_rel_dict = {"ACTION":["Actor"],
                      "CONCEPT":["Attribute", "Object"],
                      "ATTRIBUTE":["Attribute", "Object"]}  
  answer_type_dict = {"ATTRIBUTE":["OBJECT", "ACTOR"],
                      "ACTOR":["ACTOR"],
                      "ACTION":["ACTION"],
                      "OBJECT":["OBJECT"]}
  question_text = question_data["text"]
  question_scope = question_data["scope"] #general: whole text; specific: focus based question (semantic similarity)
  question_focus_concept = question_data["focus_concept"]
  answer_type = question_data["focus_feature"]
  print(question_data)
  #preprocessed_corpus = preprocess(corpus_path, nlp) **corpus_data
   
  answer = []
  if(question_scope == "general"):
    #labeling entities
    #preprocessed_corpus = ner_model.pipe(preprocessed_corpus) ** ner_corpus

    for chunk in ner_corpus:
      for entity in chunk.ents:
        if(entity.label_ in answer_type_dict[answer_type.upper()]):
          answer.append(entity.text)
  elif(question_scope == "specific"):
    candidate_docs = get_candidate_docs(nlp(question_focus_concept), nlp_corpus, 0.4)
    print(f"Candidate docs idx: \n ---> {candidate_docs}")
    relex_pp_docs = [doc for idx, doc in enumerate(relex_corpus) if idx in candidate_docs]
    ner_pp_docs = [doc for idx, doc in enumerate(ner_corpus) if idx in candidate_docs]
    for idx, doc in enumerate(ner_pp_docs):
      relationships = relex_pp_docs[idx]
      #print(relationships)
      for entity in doc.ents:
        if(entity.label_ in answer_type_dict[answer_type.upper()]):
          #Special case for functions --> (Function - Object)
          #print(f"answer_type {answer_type}")
          if(answer_type == "action"):
            for rel in relationships:
              if(rel[0]==question_focus_concept and rel[1]==entity.text):
                if(rel[2][0] in feature_rel_dict[answer_type.upper()] and rel[2][1] > 0.5):
                  #searching for function objects
                  for rel2 in relationships:
                    if(rel2[0] == entity.text and rel2[2][0] == "Object" and rel2[2][1] > 0.5):
                      func_object = rel2[1]
                      answer.append(f"{entity.text} {func_object}")
                      #answer.append((entity.text, func_object))
          elif(answer_type == "attribute"):
            rel_1_score, rel_2_score = 0,0
            for rel in relationships:
              if(rel[2][0] in feature_rel_dict[answer_type.upper()] and rel[2][1]>0.2):
                if(rel[0]==question_focus_concept and rel[1]==entity.text):
                  rel_1_score = rel[2][1]
                #Checking the opposite direction relationship between the same concepts
                #Just return it if is greater  
                elif(rel[0]==entity.text and rel[1]==question_focus_concept):
                  rel_2_score = rel[2][1]
            if(rel_1_score > rel_2_score):
              answer.append(f"{entity.text}")
          else:
            for rel in relationships:
              if(rel[0]==question_focus_concept and rel[1]==entity.text):
                #print(rel[2][0], feature_rel_dict[answer_type.upper()])
                #print(rel[2][1])
                if(rel[2][0] in feature_rel_dict[answer_type.upper()] and rel[2][1] > 0.5):
                  answer.append(entity.text)

  return set(answer)

def ask_square(question, ontology, corpus_path):
   
  ontology_structure = get_ontology_structure(ontology)
  wh_auxiliaries = {"single":"is", "multiple":"are"}
  ontology_classes = list(ontology_structure.keys())
  focus_concept, focus_feature = get_question_focus(question)
  if(focus_feature == "related concept"):
    answer = [focus_concept]
  else:
    question_data = {"text":question,
                     "scope":"general" if focus_concept == "discourse" else "specific",
                     "focus_concept":focus_concept,
                     "focus_feature":focus_feature}
    answer = extract_answer(question_data, ner_corpus, relex_corpus, nlp_corpus)
  return answer
ontology_path = "drive/MyDrive/RENER/answer_extraction/ontre.owl"
ontology = get_ontology("file://"+ontology_path).load()
question = 'what are the attributes of the "dataset"?'
corpus_path = "cask_dataset.txt"
ask_square(question, ontology, corpus_path)

In [ ]:
with open("cask_questions.txt",'r') as f:
  cask_questions = f.readlines()

def beautify_question(question):
  question = question.replace("\n","")
  len_flag = len("of the ")
  start_idx = question.index("of the ")
  end_idx = question.index("?")
  question_focus = question[start_idx+len_flag:end_idx]
  new_question = f'{question[:start_idx+len_flag]}"{question_focus}"?'
  return new_question
cask_questions = [question for question in cask_questions if question != "\n"]
cask_questions = list(map(lambda x: beautify_question(x), cask_questions))

In [ ]:
%%time
for question in cask_questions:
  answer = ask_square(question, ontology, corpus_path)
  if answer:
    for a in answer:
      print(a.lower())
  else:
    print("None")